# Distilbert experimenty

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import tensorflow as tf
from torch.utils.data import Dataset
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder
from typing import Dict, List
import transformers
from transformers import AutoModel, BertTokenizerFast,BertTokenizer, DistilBertTokenizerFast
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, AutoModelForMaskedLM

!pip install transformers[torch]
!pip install accelerate -U
!pip install transformers==4.30

## Načítanie a príprava dát

In [ ]:
dataset = pd.read_csv('../Data/final_dataset_3.csv')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset['processed_text'], dataset['author_id'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_valid = label_encoder.transform(y_valid)
y_test = label_encoder.transform(y_test)

## Distilbert

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
x_train = list(X_train)
x_val = list(X_valid)
x_test = list(X_test)

y_train = list(y_train)
y_val = list(y_valid)
y_test = list(y_test)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_val,truncation=True, padding=True)
test_encodings = tokenizer(x_test, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
train_dataset = CustomDataset(train_encodings, y_train)
val_dataset = CustomDataset(val_encodings, y_valid)
test_dataset = CustomDataset(test_encodings, y_test)

In [ ]:
labels_count = len(dataset['author_id'].unique())
print(labels_count)

20


## Experiment 1 - learning_rate = 5e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=5e-5,             #default
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.we

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.892453,0.096154,0.004808,0.050000,0.008772
2,No log,2.063524,0.519231,0.375017,0.509508,0.409545
3,No log,1.498983,0.564103,0.561397,0.568018,0.503157
4,No log,0.988622,0.666667,0.684690,0.660317,0.618406
5,No log,0.655958,0.788462,0.850421,0.772143,0.792487
6,1.489100,0.749604,0.750000,0.777982,0.734720,0.714002
7,1.489100,0.525808,0.833333,0.833872,0.821210,0.812083
8,1.489100,0.573569,0.833333,0.842910,0.825933,0.818488
9,1.489100,0.517048,0.858974,0.852020,0.848294,0.839416
10,1.489100,0.515504,0.846154,0.836399,0.837738,0.828506


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=1056, training_loss=0.7167347658594901, metrics={'train_runtime': 365.3519, 'train_samples_per_second': 46.114, 'train_steps_per_second': 2.89, 'total_flos': 2232527153725440.0, 'train_loss': 0.7167347658594901, 'epoch': 12.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-2.6015625 , -2.0039062 , -2.0683594 , ..., -1.1552734 ,
         7.3320312 , -0.28515625],
       [-0.9067383 ,  2.5625    ,  3.2519531 , ..., -1.7373047 ,
        -3.625     , -3.0292969 ],
       [-0.41430664,  6.2421875 ,  2.8222656 , ..., -2.1191406 ,
        -2.7714844 , -2.5683594 ],
       ...,
       [-3.2617188 , -2.2402344 , -2.6914062 , ..., -0.8823242 ,
         7.40625   , -0.35961914],
       [-3.0585938 , -2.4511719 , -2.7988281 , ..., -1.5546875 ,
         7.2734375 ,  2.90625   ],
       [-0.07904053, -2.1269531 , -1.3173828 , ..., -0.8803711 ,
        -1.0683594 , -0.82666016]], dtype=float32), label_ids=array([18,  1,  2, 18, 15,  9,  1, 19,  6, 15, 15, 18,  1, 12, 14,  8,  4,
       19,  9, 16, 17, 16,  7, 13,  0,  7, 15, 15, 11,  4, 17, 12,  5, 18,
       12,  7, 16,  1, 17, 19,  2,  8,  4,  2, 19,  7,  7,  7,  8,  5,  7,
       11,  8,  5, 10,  9,  9, 19, 19,  8,  2, 12, 12, 19,  6, 12, 15,  0,
       18, 12,  6,  8, 12, 16,  

## Experiment 4 - learning_rate = 8e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=8e-5,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.we

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.724722,0.262821,0.261852,0.210585,0.179880
2,No log,1.796022,0.493590,0.418654,0.483910,0.384983
3,No log,1.329270,0.564103,0.571810,0.571815,0.488038
4,No log,0.789142,0.717949,0.778405,0.710087,0.712806
5,No log,0.817502,0.756410,0.810516,0.750621,0.737211
6,1.339100,0.671228,0.807692,0.817411,0.789048,0.780852
7,1.339100,0.505248,0.852564,0.857927,0.845836,0.841680
8,1.339100,0.695992,0.839744,0.840100,0.832163,0.816087
9,1.339100,0.507218,0.878205,0.863948,0.867242,0.855741
10,1.339100,0.528370,0.871795,0.870774,0.863770,0.852838


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=1056, training_loss=0.6427387067568348, metrics={'train_runtime': 388.6569, 'train_samples_per_second': 43.349, 'train_steps_per_second': 2.717, 'total_flos': 2232527153725440.0, 'train_loss': 0.6427387067568348, 'epoch': 12.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-1.8007812 , -1.4472656 , -2.9316406 , ..., -2.9882812 ,
         7.8671875 , -0.79052734],
       [-0.21240234,  1.4023438 ,  6.6757812 , ..., -1.296875  ,
        -3.015625  , -3.5429688 ],
       [ 0.07000732,  6.9296875 ,  2.5917969 , ..., -2.1582031 ,
        -2.5605469 , -3.4296875 ],
       ...,
       [-2.6601562 , -1.8701172 , -2.671875  , ..., -3.0625    ,
         6.734375  , -1.0732422 ],
       [-2.2207031 , -1.6484375 , -3.2167969 , ..., -2.65625   ,
         8.40625   ,  0.5       ],
       [-1.1621094 , -1.4980469 , -2.3671875 , ..., -0.56933594,
        -0.29516602, -0.79052734]], dtype=float32), label_ids=array([18,  1,  2, 18, 15,  9,  1, 19,  6, 15, 15, 18,  1, 12, 14,  8,  4,
       19,  9, 16, 17, 16,  7, 13,  0,  7, 15, 15, 11,  4, 17, 12,  5, 18,
       12,  7, 16,  1, 17, 19,  2,  8,  4,  2, 19,  7,  7,  7,  8,  5,  7,
       11,  8,  5, 10,  9,  9, 19, 19,  8,  2, 12, 12, 19,  6, 12, 15,  0,
       18, 12,  6,  8, 12, 16,  